In [1]:
import pandas as pd
import numpy as np
from numpy.linalg import *
import scipy as sp
import matplotlib.pyplot as plt
import os
import seaborn as sns
from sklearn.decomposition import *
from sklearn.metrics import *
from sklearn.neighbors import *
from collections import *
from pyclust import *
from sklearn.manifold import *
from sklearn.cluster import KMeans
from module.EKmeans import *
from module.CTree import *
import datetime
from collections import *

DATA_PATH = './trade_new.csv'

_col = {
    'uid':'uid',
    'tran_time':'sldatime', #
    'gender_age':'cmrid',
    'vipno':'vipno', #
    'itemno':'pluno', #
    'amount':'amt', #
    'quantity':'qty', #
    'brandno':'bndno', #
}

_col_class = {
    'class1':'class1',
    'class2':'class2',
    'class3':'class3',
    'class4':'class4',
    'class5':'class5',
}

_col2 ={**_col,**_col_class}

print('columns',_col2)

CTree.py init
columns {'uid': 'uid', 'tran_time': 'sldatime', 'gender_age': 'cmrid', 'vipno': 'vipno', 'itemno': 'pluno', 'amount': 'amt', 'quantity': 'qty', 'brandno': 'bndno', 'class1': 'class1', 'class2': 'class2', 'class3': 'class3', 'class4': 'class4', 'class5': 'class5'}


In [2]:
df = pd.read_csv(DATA_PATH)
df = df[_col.values()]
new_df = pd.DataFrame([],columns=_col2.values())


category = df[_col2['itemno']].values.astype('str')
category = np.array(list(map(lambda x:np.array([x[:2],x[:3],x[:4],x[:5],x[5:]]),category)))
category = pd.DataFrame(category,columns=_col_class.values())
if (category.index.start == df.index.start) and \
    (category.index.stop == df.index.stop):
    new_df = df.join(category)

df_amount_sum = new_df.groupby([_col2['vipno'],_col2['class1'],_col2['class2'],_col2['class3'],_col2['class4']],as_index=False).agg({
    _col2['amount']:sum
})
df_amount_sum3 = new_df.groupby([_col2['vipno'],_col2['class1'],_col2['class2'],_col2['class3']],as_index=False).agg({
    _col2['amount']:sum
})
df_amount_sum2 = new_df.groupby([_col2['vipno'],_col2['class1'],_col2['class2']],as_index=False).agg({
    _col2['amount']:sum
})
df_amount_sum1 = new_df.groupby([_col2['vipno'],_col2['class1']],as_index=False).agg({
    _col2['amount']:sum
})
df_amount = [df_amount_sum1, df_amount_sum2, df_amount_sum3, df_amount_sum]
del df
# df_amount_sum.to_csv('./result/a1-amount_sum.csv')

### problem 1 2

In [27]:
set_vipno = set(df_amount_sum[_col2['vipno']])
mapping_index_v = {i:idx for (idx,i) in zip(range(len(set_vipno)),set_vipno)}


def aggByUser(category='class4'):
    global set_vipno,df_amount,_col2
    map_single = {v:{} for v in set_vipno}
    df_temp = df_amount[ int(category[-1])-1 ]
    for v in set_vipno:
        lst_v = df_temp.loc[ df_temp[_col2['vipno']]==v ]
        for idx,row in lst_v.iterrows():
            map_single[v][ row[ _col2[ category ] ] ] = row[ _col2['amount'] ]
    return map_single


def computeUserFeatureOfClass(category='class4'):
    global df_amount,mapping_index_v,_col2
    set_class_i = set(new_df[_col2[ category ]])
    mapping_index_class_i = {i:idx for (idx,i) in zip(range(len(set_class_i)),set_class_i)}
    mat_userFeature = np.array([[0.0 for i in range(len(set_class_i))] for j in range(len(set_vipno))])
    for idx,r in df_amount[ int(category[-1])-1 ].iterrows():
        mat_userFeature[ mapping_index_v[ r[ _col2['vipno'] ] ] ][ mapping_index_class_i[ r[ _col2[category] ] ] ] = r[ _col2['amount'] ]
    return mat_userFeature


def computeJaccardDistance(a,b):
    return 1-np.sum(np.min([a,b],0))/np.sum(np.max([a,b],0))


def computeJaccardMatrix(mat_userFeature):
    global set_vipno,map_single,mapping_index_v,D
    mat_jaccard = np.array([[0 for i in range(len(set_vipno))] for j in range(len(set_vipno))]).astype('float')
    for v1 in set_vipno:
        for v2 in set_vipno:
            jaccard_pairwise_dist = compute_distance(a=mat_userFeature[mapping_index_v[v1]],b=mat_userFeature[mapping_index_v[v2]],D=D,metric='cos')
            mat_jaccard[mapping_index_v[v1]][mapping_index_v[v2]] = mat_jaccard[mapping_index_v[v2]][mapping_index_v[v1]] = jaccard_pairwise_dist
    return mat_jaccard

def compute_distance(a,b,D,metric='jaccard'):
    if metric=='jaccard':
        return computeJaccardDistance(a,b)
    elif metric=='euclidean':
        return norm(a-b)
    elif metric=='cos':
        score = [computeJaccardDistance(a[:D[0]],b[:D[0]]),
                computeJaccardDistance(a[D[0]:D[1]],b[D[0]:D[1]]),
                computeJaccardDistance(a[D[1]:D[2]],b[D[1]:D[2]]),
                computeJaccardDistance(a[D[2]:D[3]],b[D[2]:D[3]])]
#             print(score)
        return sum(score)/4

class EKmeans():
    def __init__(self,X,D,method='jaccard',n_centers=2,max_iters=1000,tol=1e-20,verbose=0):
        self.n_centers = n_centers
        self.max_iters = max_iters
        self.D = D
        self.method = method
        self.X = X
        self.clusters = np.array([-1 for i in range(X.shape[0])])
        self.centers = self.X[np.random.permutation(X.shape[0])[:self.n_centers]]
        self.pre_centers = []
        self.debug = []
        self.lr = 0.34
        self.tol = tol
        self.verbose = verbose
        return
    
    def metrics_compactness(self):
        s = 0.0
        for c in range(self.n_centers):
            idx_cluster = np.where(self.clusters==c)[0]
            for a in self.X[ idx_cluster ]:
                dst = compute_distance(a=a,b=self.centers[c],D=self.D,metric=self.method)
                s += dst*1.0/len(idx_cluster)
        return s
                
    def __update_centers(self,smooth=True):
        self.pre_centers = np.array(self.centers)
        for c in range(self.n_centers):
            temp = self.X[ np.where(self.clusters==c)[0] ]
            if len(temp):
                self.centers[c] = np.array([ np.mean( temp, 0 ) ])
#             print(self.centers,self.X[ np.where(self.clusters==c)[0] ]) ### debug
        if smooth:
            self.centers = (self.centers-self.pre_centers)*self.lr + self.pre_centers
        return
    
    def __log(self,x):
        if self.verbose:
            print(x)
        return
    
    def fit(self):
        for kiter in range(self.max_iters):
            self.__log('iteration %d'%kiter)
            for i in range(self.X.shape[0]):
                d_min_dist = 1e10
                record_node = -1
                for idx,c in zip(range(len(self.centers)),self.centers):
                    dst = compute_distance(a=self.X[i],b=c,D=self.D,metric=self.method)
                    if dst<d_min_dist:
                        d_min_dist = dst
                        record_node = idx
                self.clusters[i] = record_node
            self.__update_centers()
            move_dist = np.mean(norm(self.pre_centers-self.centers,axis=1))
            self.debug.append(move_dist)
            if (kiter - kiter//20 *20 ==1) and move_dist<self.tol:
                break
            if kiter==10:
#                 raise Exception('stop')
                pass
        return


# map_single = aggByUser()
mat_userFeature = [computeUserFeatureOfClass('class1'),
                   computeUserFeatureOfClass('class2'),
                   computeUserFeatureOfClass('class3'),
                   computeUserFeatureOfClass('class4')
                  ]

D =np.cumsum(
    [
        mat_userFeature[0].shape[1],
        mat_userFeature[1].shape[1],
        mat_userFeature[2].shape[1],
        mat_userFeature[3].shape[1]
    ]
)

mat_jaccard = computeJaccardMatrix(np.hstack(mat_userFeature))

def KmeansCluster(n_centers,method='cos',max_iters=500):
    global mat_userFeature,D
    if method=='jaccard':
        kmeans = EKmeans(mat_userFeature[-1],None,n_centers=n_centers,max_iters=max_iters)
    elif method=='cos':
        kmeans = EKmeans(np.hstack(mat_userFeature),D,n_centers=n_centers,max_iters=max_iters,method='cos')
    elif method=='euclidean':
        kmeans = EKmeans(mat_userFeature[-1],None,n_centers=n_centers,max_iters=max_iters,method='euclidean')
    kmeans.fit()
    return kmeans


kmeans = KMeans(n_clusters=4)
clusters = kmeans.fit_predict(mat_userFeature[-1])
silhouette_score(mat_userFeature[-1],labels=clusters)
kmeans.cluster_centers_

In [ ]:
res = []
for nn in range(2,20):
    print('param ',nn)
    kmeans = KmeansCluster(nn,'cos',1000)
    sil = silhouette_score(mat_jaccard,labels=kmeans.clusters,metric='precomputed')
    com = kmeans.metrics_compactness() 
    res.append({'sil':sil,'com':com,'nn':nn})

# tsne = TSNE(n_jobs=-1,method='exact',n_components=2)
# mat_userFeature_t = tsne.fit_transform(np.hstack(mat_userFeature))

# sns.set(style="ticks")
# sns.pairplot(pd.DataFrame(mat_userFeature[:,100:110]),kind='reg')

### problem 3

In [3]:
### 将时间转化为 time level 1-4
def processTimeLevel(new_df):
    global _col2
    _col2.update({
        'tran_time_level':'tran_time_level',
        'tran_date':'tran_date'
    })
    
    tran_time = new_df[ _col2['tran_time'] ]
    tran_time = list(map(lambda x:datetime.datetime.strptime(x,'%Y-%m-%d %H:%M:%S').timestamp(),tran_time))
    new_df[ _col2['tran_time'] ] = tran_time
    
    tran_date = list( map(lambda x:datetime.datetime.strftime(datetime.datetime.fromtimestamp(x),'%Y%m%d'),tran_time) ) ### to y-m-d
    new_df[ _col2['tran_date'] ] = tran_date
    
    df_temp = new_df[ [_col2['vipno'], _col2['tran_date']] ]
#     df_temp = df_temp.groupby(by=['vipno']).agg({_col2['tran_date']:max}).reset_index().values
#     df_temp = {x[0]:x[1] for x in df_temp}
    q = max(df_temp[ _col2['tran_date'] ])
    df_temp = {row['vipno']:q for idx,row in df_temp.iterrows()}
#     print(set(new_df['vipno'])-set(df_temp.keys()))
    tran_time_level = []
    for idx,row in new_df.iterrows():
        latest_date = q
        latest_date = datetime.datetime.strptime(latest_date,'%Y%m%d').timestamp()
        latest_date = datetime.datetime.fromtimestamp(latest_date)
        time_level4 = (latest_date - datetime.timedelta(days=30)).timestamp()
        time_level3 = (latest_date - datetime.timedelta(days=2*30)).timestamp()
        time_level2 = (latest_date - datetime.timedelta(days=4*30)).timestamp()
#         time_level1 = (latest_date - datetime.timedelta(days=16*30)).timestamp()
        time_levels = np.array([time_level2,time_level3,time_level4])
        row_time = row[_col2['tran_time']]
        tran_time_level.append( len(np.where(row_time>time_levels)[0])+1 ) ### append time level
        
    new_df[ _col2['tran_time_level'] ] = tran_time_level
    return new_df


new_df = processTimeLevel(new_df)
new_df = new_df.sort_values(by=[_col2['vipno']]) ### sort by customer



In [4]:

#### create single tree by one transaction row
def createTreeBySingleTrans(row):
    troot = TreeNode(nodeType='root')
#     root.display()
    vipno = row[ _col2['vipno'] ]
    class1 = row[_col2['class1']]
    class2 = row[_col2['class2']]
    class3 = row[_col2['class3']]
    class4 = row[_col2['class4']]
    tran_date = row[_col2['tran_date']]
    newnode = TreeNode(nodeType='innode',nodeClass=class1,level=1,freq=1,tm=tran_date,parent=troot)
    troot.att_childs.append(newnode)
    cur_node = troot.att_childs[0]
    for idx,c in enumerate([class2,class3,class4],2):
#         cur_node.out()
        if idx==4:
            nodeType='leaf'
            f=0
        else:
            if idx==row[_col2['tran_time_level']]:
                nodeType = 'leaf'
                f=0
            else:
                f=1
                nodeType='innode'
        newnode = TreeNode(nodeType=nodeType,nodeClass=c,level=idx,freq=1,tm=tran_date,parent=cur_node)
        cur_node.att_childs.append(newnode)
        cur_node = cur_node.att_childs[0]
        if f==0:
            break
    return {'vipno':vipno,'root':troot}
    

def mergeNode(node1,node2,parent_node,tag='union'):
    if node1.att_class == node2.att_class and \
        node1.att_level == node2.att_level:
        if 'leaf' in [node1.node_type,node2.node_type]:
            node_type = 'innode' if tag=='union' else 'leaf'
        else:
            node_type = 'innode'
#             if node1.node_type!=node2.node_type:
#                 raise Exception('error',node1.out(),node2.out())
        return TreeNode(nodeType=node1.node_type,
                        nodeClass=node1.att_class,
                        level=node1.att_level,
                        freq=sum([node1.att_freq,node2.att_freq]),
                        tm=str(max(int(node1.att_tm),int(node2.att_tm))),
                        parent=parent_node)
    else:
        raise Exception('error',node1.out(),node2.out())
        return

    
def findNodeByClass(root,nodeClass,level=4):
    res = [root]
    ### get all node
    for i in range(level):
        temp = []
        [temp.extend(x.att_childs) for x in res]
        res.extend(temp)
    res = list(filter(lambda x:x.att_class==nodeClass,res)) ### find the node
    if len(res):
        return res[0]
    else:
#         print('not found class',nodeClass)
        raise Warning('not found class')
        return None
        
        
def unionTreeOfCusotomer(root1,root2,vipno=False):
    res = {}
    root = TreeNode(nodeType='root')
    res['tree'] = root
    
    if vipno:
        res['vipno'] = vipno
    try:
        current_level = []
        [current_level.extend(x.att_childs) for x in [root1,root2]]
    except Exception as e:
        print(e)
        root1.display()
        root2.display()
        raise e
        
    next_level = []
    for i in range(1,5):
        level = i
        class_counter = Counter([x.att_class for x in current_level])
        class_intersect = list(filter(lambda x:class_counter[x]>1,class_counter))
        class_non_intersect = list(filter(lambda x:class_counter[x]==1,class_counter))
        while len(current_level):
            node = current_level.pop(0)
            if node.att_class in class_intersect:
                parent_node = findNodeByClass(root=root,nodeClass=node.att_parent.att_class,level=node.att_parent.att_level)
                if parent_node is None:
                    node.out()
                    raise Exception('no parent')
                node2 = list(filter(lambda x:x.att_class==node.att_class,current_level))[0]
                current_level.remove(node2)
                parent_node.att_childs.append(mergeNode(node,node2,parent_node))
                next_level.extend(node.att_childs)
                next_level.extend(node2.att_childs)
            else:
                parent_node = findNodeByClass(root=root,nodeClass=node.att_parent.att_class,level=node.att_parent.att_level)
                parent_node.att_childs.append(node)
                node.att_parent = parent_node
        current_level = next_level
        next_level = []
         
    return res


def intersectTreeOfCusotomer(root1,root2,vipno=False):
    res = {}
    root = TreeNode(nodeType='root')
    res['tree'] = root
    
    if vipno:
        res['vipno'] = vipno
    
    current_level = []
    [current_level.extend(x.att_childs) for x in [root1,root2]]
    next_level = []
    for i in range(1,5):
        level = i
        class_counter = Counter([x.att_class for x in current_level])
        class_intersect = list(filter(lambda x:class_counter[x]>1,class_counter))
        class_non_intersect = list(filter(lambda x:class_counter[x]==1,class_counter))
        while len(current_level):
            node = current_level.pop(0)
            if node.att_class in class_intersect:
                parent_node = findNodeByClass(root=root,nodeClass=node.att_parent.att_class,level=node.att_parent.att_level)
                node2 = list(filter(lambda x:x.att_class==node.att_class,current_level))[0] ### get node to be merged
                current_level.remove(node2)
                parent_node.att_childs.append(mergeNode(node,node2,parent_node,tag='intersect')) ### add node
                next_level.extend(node.att_childs)
                next_level.extend(node2.att_childs)
        current_level = next_level
        next_level = []
         
    return res


def buildTreeOfCustomers(df,all_customers):
    global _col2
    customer_forest = {c:TreeNode(nodeType='root') for c in all_customers}
    for idx,row in df.iterrows():
        cust = row[ _col2['vipno'] ]
        subtree = createTreeBySingleTrans(row)['root']
        customer_forest[cust] = unionTreeOfCusotomer(customer_forest[cust],subtree)['tree']
    return customer_forest


def sim_v_ur(v,ur):
    try:
        parentnode_ur = findNodeByClass(root=ur,nodeClass=v.att_parent.att_class)
    except Warning as e:
        return 0
#     if parentnode_ur is None:
#         return 0
    freq_ur = [x.att_freq for x in parentnode_ur.att_childs]
    try:
        res = float(v.att_freq/np.sum(freq_ur))
        return res
    except Exception as e:
        print(e)
        raise e

def sim_vl_ur(vl,ur):
    if len(vl)==0:
        return 0
    try:
        s = list(map(lambda x:sim_v_ur(x,ur),vl))
        return np.mean(s)
    except Exception as e:
        print(e)
        raise e
        
def dist_ir_ur(ir,ur):
    utree = unionTreeOfCusotomer(ir,ur)['tree']
    itree = intersectTreeOfCusotomer(ir,ur)['tree']
    ir = itree
    ur = utree
    nodes_ir = ir.get_all_nodes()
    if len(nodes_ir)==0:
        return 1
    H = max(set(list(map(lambda x:x.att_level,nodes_ir))))
    s = 0
    for i in range(1,H+1):
        w = i/np.sum(range(1,H+1))
        vl = list(filter(lambda x:x.att_level==i,nodes_ir))
        s += w*sim_vl_ur(vl,ur)
    return 1-s

def updateTreeByFrequence(root,freq):
    current_level = [x for x in root.att_childs]
    while len(current_level):
        x = current_level.pop(0)
        if x.att_freq>=freq:
            current_level.extend(x.att_childs)
        else:
            try:
                p = x.att_parent
                p.att_childs.remove(x)

#                 temp_node = p
#                 while temp_node.node_type!='root':
#                     temp_node.att_freq -= x.att_freq
#                     temp_node = temp_node.att_parent

            except Exception as e:
                print(e)
                return x

    return root

def getClusterCentroid(forest_customer):
    global customers
    centroid = TreeNode(nodeType='root')
    for tr in forest_customer:
        centroid = unionTreeOfCusotomer(centroid,forest_customer[tr])['tree']
    
    current_freq = 1
    mindist = 1e10
    freq_step = centroid.get_avg_freq()
    freq_end = centroid.get_max_freq()
    centroid_record = centroid
    size_average = np.mean(list(map(lambda x:forest_customer[x].get_node_size(),forest_customer)))
    print('average node',size_average)
    
    while current_freq<freq_end:
        centroid = updateTreeByFrequence(centroid,current_freq)
        dist = sum([dist_ir_ur(forest_customer[c],centroid) for c in forest_customer])
        if dist<mindist:
            mindist = dist
            centroid_record = centroid
        if centroid.get_node_size()<size_average:
            break
        current_freq += freq_step
    return centroid_record

    

def clusterMain():
    
    ### get distant pair
    def distantPair(topK_key,forest_concern):
        max_dist = 0
        max_pair = None
        for i in topK_key:
            for j in topK_key:
                if j>i:
                    dist = dist_ir_ur(forest_concern[i],forest_concern[j])
                    if dist>max_dist:
                        max_dist = dist
                        max_pair = (i,j)
        return [max_dist,max_pair]

    ### recluster forest_concern
    def recluster():
        subcluster = [[],[]]
        for c in forest_concern:
            current_tree = forest_concern[c]
            [dist1,dist2] = [
                dist_ir_ur(current_tree,forest_concern[max_pair[0]]),
                dist_ir_ur(current_tree,forest_concern[max_pair[1]])
            ]
            if dist1<dist2:
                subcluster[0].append(c)
            else:
                subcluster[1].append(c)
        return subcluster

    ### precluster [] ; postcluster [[]]
    def computeBIC(precluster,postcluster):

        N = len(precluster)
        D = sum([customer_buy_count[x] for x in precluster])
        pre_coff_1 = (N-1)
        pre_sigma_s = 1.0/pre_coff_1 * np.sum(np.power(list(dist_concern.values()),2))
        pre_likelihood = -N/2*np.log(2*np.pi) - N*D/2*np.log(pre_sigma_s) - pre_coff_1/2
        pre_bic = pre_likelihood - 1*(D+1)/2*np.log(1)

        N_i = np.array([len(postcluster[0]), len(postcluster[1])])
        D_i = np.array([
            sum([customer_buy_count[x] for x in postcluster[0]]),
            sum([customer_buy_count[x] for x in postcluster[1]])
        ])
        post_coff_1 = N_i - 2
        post_sigma_s = 1.0/post_coff_1 * np.array([ 
            np.sum( np.power(list( dist_1.values() ),2) ), 
            np.sum( np.power(list( dist_2.values() ),2) )
        ])
        post_likelihood = N_i*np.log(N_i) - N_i*np.log(N) - N_i/2*np.log(2*np.pi) - N_i*D/2*np.log(post_sigma_s) - post_coff_1/2
        post_likelihood = np.sum(post_likelihood)
        post_bic = post_likelihood - 2*(D+1)/2*np.log(2)

        print('bic ',pre_bic,post_bic)
        return pre_bic,post_bic
    
    ### compute metrics and scores
    def computeMetrics(cluster_final):
        cluster_final = cluster_static
        mapping = {x:i for i,x in enumerate(customers,0)}
        d = np.array([[0.0 for i in range(len(customers))] for j in range(len(customers))])
        for i,x in enumerate(customers,0):
            for j,y in enumerate(customers,0):
                if j>=i:
                    d[ mapping[x] ][ mapping[y] ] = dist_ir_ur(forest_customer[x],forest_customer[y])
                    d[ mapping[y] ][ mapping[x] ] = d[ mapping[x] ][ mapping[y] ]

        sc = 0
        B = np.array([0.0 for i in range(len(customers))])
        A = np.array([0.0 for j in range(len(customers))])
        cust_all = set(customers)
        for cl in cluster_final:
            cust_cl = set(cl)
            for p in cl:
                same = cust_cl - set([p])
                other = cust_all - cust_cl - set([p])
                A[ mapping[p] ] = np.mean([d[ mapping[p] ][ mapping[s] ] for s in same])
                B[ mapping[p] ] = np.mean([d[ mapping[p] ][ mapping[o] ] for o in other])
        sc = np.mean([(B[i]-A[i])/np.max([A[i],B[i]]) for i in range(len(customers))])
        print('sc ',sc)

        cp = np.mean([ 
            np.mean( list(map(lambda x:dist_ir_ur(forest_customer[x],centroid_final[i]),cl)) ) 
            for i,cl in enumerate(cluster_final,0) 
        ])
        print('cp ',cp)

        return sc,cp

    top_ratio = 0.5
    topK_max = 50
    split_threshold = 20
    global new_df
    customer_buy_count = new_df.groupby(by=[_col2['vipno']]).count()['uid'].to_dict() ### { customer:buy_count }
    customers = list( set(new_df[_col2['vipno']]) ) ### all customers : list
    forest_customer = buildTreeOfCustomers(new_df,customers) ### customer tree {vipno:root}
    cluster_dynamic = [[x for x in customers]]
    cluster_static = []
    centroid_final = []
    while len(cluster_dynamic):
        cluster_concern = cluster_dynamic.pop(0)
        cluster_concern = set(cluster_concern)
        forest_concern = list(filter(lambda x:x in cluster_concern,forest_customer))
        forest_concern = { x:forest_customer[x] for x in forest_concern }
        centroid_concern = getClusterCentroid(forest_concern) ### centoid tree of interest
        dist_concern = { x:dist_ir_ur(forest_concern[x],centroid_concern) for x in forest_concern } ### dist of @forest_concern & @centroid_concern
        topK = min( [ topK_max,max([2,top_ratio*len(forest_concern)]) ] )
        topK = int(topK)
        topK_key = sorted(dist_concern,key=lambda x:dist_concern[x],reverse=True)[:topK]
        [max_dist,max_pair] = distantPair(topK_key,forest_concern) ### find most distant pair
        subcluster = recluster()
        subforest_1 = { x:forest_customer[x] for x in subcluster[0] }
        subforest_2 = { x:forest_customer[x] for x in subcluster[1] }
        centroid_1 = getClusterCentroid(subforest_1) ### costing
        centroid_2 = getClusterCentroid(subforest_2) ### costing
        dist_1 = { x:dist_ir_ur(subforest_1[x],centroid_1) for x in subcluster[0] } ### costing
        dist_2 = { x:dist_ir_ur(subforest_2[x],centroid_2) for x in subcluster[1] } ### costing
        [ pre_bic,post_bic ] = computeBIC(list(cluster_concern),subcluster)
        if post_bic<pre_bic or (len(subforest_1)<split_threshold or len(subforest_2)<split_threshold):
            cluster_static.append(list(cluster_concern))
            centroid_final.append(centroid_concern)
            print('settle ',len(cluster_concern))
        else:
            cluster_dynamic.extend(subcluster)
            print('split ',len(subcluster[0]),len(subcluster[1]))
    return cluster_static

test

In [ ]:
# del root1
root1 = createTreeBySingleTrans(new_df.iloc[0])
root2 = createTreeBySingleTrans(new_df.iloc[1])
root3 = createTreeBySingleTrans(new_df.iloc[2])
root12 = unionTreeOfCusotomer(root1['root'],root2['root'])['tree']
# unionTreeOfCusotomer(root12,root3['root'])['tree'].display()
intersectTreeOfCusotomer(root12,root12)['tree'].display()

In [5]:
### get distant pair
def distantPair(topK_key,forest_concern):
    max_dist = 0
    max_pair = None
    for i in topK_key:
        for j in topK_key:
            if j>i:
                dist = dist_ir_ur(forest_concern[i],forest_concern[j])
                if dist>max_dist:
                    max_dist = dist
                    max_pair = (i,j)
    return [max_dist,max_pair]

### recluster forest_concern
def recluster():
    subcluster = [[],[]]
    for c in forest_concern:
        current_tree = forest_concern[c]
        [dist1,dist2] = [
            dist_ir_ur(current_tree,forest_concern[max_pair[0]]),
            dist_ir_ur(current_tree,forest_concern[max_pair[1]])
        ]
        if dist1<dist2:
            subcluster[0].append(c)
        else:
            subcluster[1].append(c)
    return subcluster

### precluster [] ; postcluster [[]]
def computeBIC(precluster,postcluster):

    N = len(precluster)
    D = sum([customer_buy_count[x] for x in precluster])
    pre_coff_1 = (N-1)
    pre_sigma_s = 1.0/pre_coff_1 * np.sum(np.power(list(dist_concern.values()),2))
    pre_likelihood = -N/2*np.log(2*np.pi) - N*D/2*np.log(pre_sigma_s) - pre_coff_1/2
    pre_bic = pre_likelihood - 1*(D+1)/2*np.log(1)

    N_i = np.array([len(postcluster[0]), len(postcluster[1])])
    D_i = np.array([
        sum([customer_buy_count[x] for x in postcluster[0]]),
        sum([customer_buy_count[x] for x in postcluster[1]])
    ])
    post_coff_1 = N_i - 2
    post_sigma_s = 1.0/post_coff_1 * np.array([ 
        np.sum( np.power(list( dist_1.values() ),2) ), 
        np.sum( np.power(list( dist_2.values() ),2) )
    ])
    post_likelihood = N_i*np.log(N_i) - N_i*np.log(N) - N_i/2*np.log(2*np.pi) - N_i*D/2*np.log(post_sigma_s) - post_coff_1/2
    post_likelihood = np.sum(post_likelihood)
    post_bic = post_likelihood - 2*(D+1)/2*np.log(2)

    print('bic ',pre_bic,post_bic)
    return pre_bic,post_bic

### compute metrics and scores
def computeMetrics(cluster_final):
    cluster_final = cluster_static
    mapping = {x:i for i,x in enumerate(customers,0)}
    d = np.array([[0.0 for i in range(len(customers))] for j in range(len(customers))])
    for i,x in enumerate(customers,0):
        for j,y in enumerate(customers,0):
            if j>=i:
                d[ mapping[x] ][ mapping[y] ] = dist_ir_ur(forest_customer[x],forest_customer[y])
                d[ mapping[y] ][ mapping[x] ] = d[ mapping[x] ][ mapping[y] ]

    sc = 0
    B = np.array([0.0 for i in range(len(customers))])
    A = np.array([0.0 for j in range(len(customers))])
    cust_all = set(customers)
    for cl in cluster_final:
        cust_cl = set(cl)
        for p in cl:
            same = cust_cl - set([p])
            other = cust_all - cust_cl - set([p])
            A[ mapping[p] ] = np.mean([d[ mapping[p] ][ mapping[s] ] for s in same])
            B[ mapping[p] ] = np.mean([d[ mapping[p] ][ mapping[o] ] for o in other])
    sc = np.mean([(B[i]-A[i])/np.max([A[i],B[i]]) for i in range(len(customers))])
    print('sc ',sc)

    cp = np.mean([ 
        np.mean( list(map(lambda x:dist_ir_ur(forest_customer[x],centroid_final[i]),cl)) ) 
        for i,cl in enumerate(cluster_final,0) 
    ])
    print('cp ',cp)

    return sc,cp

top_ratio = 0.5
topK_max = 50
split_threshold = 15
global new_df
customer_buy_count = new_df.groupby(by=[_col2['vipno']]).count()['uid'].to_dict() ### { customer:buy_count }
customers = list( set(new_df[_col2['vipno']]) ) ### all customers : list
forest_customer = buildTreeOfCustomers(new_df,customers) ### customer tree {vipno:root}
cluster_dynamic = [[x for x in customers]]
cluster_static = []
centroid_final = []
while len(cluster_dynamic):
    cluster_concern = cluster_dynamic.pop(0)
    cluster_concern = set(cluster_concern)
    forest_concern = list(filter(lambda x:x in cluster_concern,forest_customer))
    forest_concern = { x:forest_customer[x] for x in forest_concern }
    centroid_concern = getClusterCentroid(forest_concern) ### centoid tree of interest
    dist_concern = { x:dist_ir_ur(forest_concern[x],centroid_concern) for x in forest_concern } ### dist of @forest_concern & @centroid_concern
    topK = min( [ topK_max,max([2,top_ratio*len(forest_concern)]) ] )
    topK = int(topK)
    topK_key = sorted(dist_concern,key=lambda x:dist_concern[x],reverse=True)[:topK]
    [max_dist,max_pair] = distantPair(topK_key,forest_concern) ### find most distant pair
    subcluster = recluster()
    subforest_1 = { x:forest_customer[x] for x in subcluster[0] }
    subforest_2 = { x:forest_customer[x] for x in subcluster[1] }
    centroid_1 = getClusterCentroid(subforest_1) ### costing
    centroid_2 = getClusterCentroid(subforest_2) ### costing
    dist_1 = { x:dist_ir_ur(subforest_1[x],centroid_1) for x in subcluster[0] } ### costing
    dist_2 = { x:dist_ir_ur(subforest_2[x],centroid_2) for x in subcluster[1] } ### costing
    [ pre_bic,post_bic ] = computeBIC(list(cluster_concern),subcluster)
    if post_bic<pre_bic or (len(subforest_1)<split_threshold or len(subforest_2)<split_threshold):
        cluster_static.append(list(cluster_concern))
        centroid_final.append(centroid_concern)
        print('settle ',len(cluster_concern))
    else:
        cluster_dynamic.extend(subcluster)
        print('split ',len(subcluster[0]),len(subcluster[1]))
computeMetrics(cluster_static)

average node 55.483539094650205
average node 52.7942238267148
average node 59.047846889952154
bic  5893241.126106286 6070024.388103115
split  277 209
average node 52.7942238267148
average node 54.40952380952381
average node 51.80813953488372
bic  1939891.0873797485 2011069.533117923
split  105 172
average node 59.047846889952154
average node 58.38505747126437
average node 62.34285714285714
bic  1161210.0586859067 1193439.483759727
split  174 35
average node 54.40952380952381
average node 59.5
average node 51.971830985915496
bic  327136.74073899403 338751.0560665581
split  34 71
average node 51.80813953488372
average node 60.1875
average node 48.564516129032256
bic  728468.194493893 758043.5849672941
split  48 124
average node 58.38505747126437
average node 56.85227272727273
average node 59.95348837209303
bic  814990.4274023339 838142.2130305989
split  88 86
average node 62.34285714285714
average node 58.1875
average node 65.84210526315789
bic  38716.98410254362 36643.01691005982
settle

(0.08408521767308817, 0.4901503310115557)

In [ ]:
# forest_customer[1591015476700].display()
computeMetrics(cluster_static)

In [14]:
#### test case 1
r1 = TreeNode(nodeType='root')
r11=TreeNode(nodeType='innode',nodeClass='Food',freq=2,level=1,parent=r1)
r1.att_childs.append(r11)
r12=TreeNode(nodeType='innode',nodeClass='Clothes',freq=1,level=1,parent=r1)
r1.att_childs.append(r12)

r21=TreeNode(nodeType='innode',nodeClass='Drink',freq=2,level=2,parent=r11)
r11.att_childs.append(r21)
r22=TreeNode(nodeType='innode',nodeClass='Children Clothes',freq=1,level=2,parent=r12)
r12.att_childs.append(r22)

r31=TreeNode(nodeType='innode',nodeClass='Tea',freq=2,level=3,parent=r21)
r21.att_childs.append(r31)

r41=TreeNode(nodeType='innode',nodeClass='Black Tea',freq=1,level=4,parent=r31)
r31.att_childs.append(r41)
r42=TreeNode(nodeType='innode',nodeClass='Green Tea',freq=1,level=4,parent=r31)
r31.att_childs.append(r42)



t1 = TreeNode(nodeType='root')
t11=TreeNode(nodeType='innode',nodeClass='Food',freq=2,level=1,parent=t1)
t1.att_childs.append(t11)
t12=TreeNode(nodeType='innode',nodeClass='Electronics',freq=1,level=1,parent=t1)
t1.att_childs.append(t12)

t21=TreeNode(nodeType='innode',nodeClass='Drink',freq=2,level=2,parent=t11)
t11.att_childs.append(t21)

t31=TreeNode(nodeType='innode',nodeClass='Tea',freq=1,level=3,parent=t21)
t21.att_childs.append(t31)
t32=TreeNode(nodeType='innode',nodeClass='Juice',freq=1,level=3,parent=t21)
t21.att_childs.append(t32)

t41=TreeNode(nodeType='innode',nodeClass='Black Tea',freq=1,level=4,parent=r31)
t31.att_childs.append(t41)

In [17]:
a = unionTreeOfCusotomer(r1,t1)['tree']
print()
b = intersectTreeOfCusotomer(r1,t1)['tree']
dist_ir_ur(r1,t1)

0.2416666666666667

In [24]:
item = new_df.iloc[0]
createTreeBySingleTrans(item)['root'].display()
new_df.head(1)

{'nodetype': 'root', 'att_childs': 1, 'att_freq': 0, 'att_class': '', 'att_level': 0, 'att_tm': 0, 'parent': ''}
{'nodetype': 'innode', 'att_childs': 1, 'att_freq': 1, 'att_class': '15', 'att_level': 1, 'att_tm': '20160730', 'parent': ''}
{'nodetype': 'innode', 'att_childs': 1, 'att_freq': 1, 'att_class': '151', 'att_level': 2, 'att_tm': '20160730', 'parent': '15'}
{'nodetype': 'innode', 'att_childs': 1, 'att_freq': 1, 'att_class': '1512', 'att_level': 3, 'att_tm': '20160730', 'parent': '151'}
{'nodetype': 'leaf', 'att_childs': 0, 'att_freq': 1, 'att_class': '15120', 'att_level': 4, 'att_tm': '20160730', 'parent': '1512'}


,uid,sldatime,cmrid,vipno,pluno,amt,qty,bndno,class1,class2,class3,class4,class5,tran_date,tran_time_level
16691,16073014031928627,1.469859e+09,女[18 - 25],781924,15120000,15.0,6.0,15012.0,15,151,1512,15120,000,20160730,4


In [ ]:
r = list(dist_concern.values())
r = sorted(r)
sns.distplot(r,bins=200)
r1 = list(dist_1.values())
r1 = sorted(r1)
r2 = list(dist_2.values())
r2 = sorted(r2)
sns.distplot(r1,bins=100)
sns.distplot(r2,bins=100)

In [ ]:
2*[1,2]